In [12]:
# -------------------------------------------------
# 1️⃣  БАЗОВАЯ НАСТРОЙКА
# -------------------------------------------------
import pandas as pd
pd.set_option('display.float_format', '{:,.2f}'.format)   # красивый вывод

# ----------------------------------------------------------------------
# 2️⃣  ЧТЕНИЕ CSV (английские заголовки, русская строка‑подпись – пропускаем)
# ----------------------------------------------------------------------
df = pd.read_csv(
    "seller_data.csv",
    header=0,              # первая строка – нужные названия колонок
    skiprows=[1],          # вторая строка (русская) отбрасываем
    na_values=[""],        # пустые ячейки → NaN
    encoding="utf-8-sig"
)

# ----------------------------------------------------------------------
# 3️⃣  ОЧИСТКА И ПРИВЕДЕНИЕ ТИПОВ
# ----------------------------------------------------------------------
# 3.1  Удаляем «итоги» внизу файла (строки без Seller_ID)
df["Seller_ID"] = pd.to_numeric(df["Seller_ID"], errors="coerce")
df = df.dropna(subset=["Seller_ID"]).reset_index(drop=True)

# 3.2  Функция очистки булевых колонок (без FutureWarning)
def clean_bool(col):
    """Удалить пробелы, привести к верхнему регистру и получить bool."""
    col = col.astype(str).str.strip().str.upper()
    mapping = {"TRUE": True, "FALSE": False, "": False, "NAN": False}
    # map → NaN для неизвестных значений, потом заполняем False
    col = col.map(mapping).fillna(False).astype(bool)
    return col

for c in ["Promotion_Used", "Subscription"]:
    df[c] = clean_bool(df[c])

# 3.3  Численные столбцы → float, отсутствующее → 0
num_cols = ["Order_Value", "Commission_Rate",
            "Promotion_Income", "Subscription_Income"]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce").fillna(0)

# ----------------------------------------------------------------------
# 4️⃣  БАЗОВЫЕ МЕТРИКИ ПЛАТФОРМЫ (из таблицы)
# ----------------------------------------------------------------------
# 4.1  Средний чек (Average Order Value)
avg_check = df["Order_Value"].mean()

# 4.2  Комиссия за одну продажу
df["Commission"] = df["Order_Value"] * df["Commission_Rate"]
avg_commission = df["Commission"].mean()

# 4.3  Доход продавца:
#      • комиссия (по всем его заказам)
#      • 1 разовый доход от платного продвижения (если использовал)
#      • 1 разовый доход от подписки (если есть)
seller_agg = (
    df.groupby("Seller_ID")
      .agg({
          "Commission": "sum",               # всё, что получено в виде комиссии
          "Promotion_Income": "max",        # берём один раз (max = либо 0, либо 5000)
          "Subscription_Income": "max"      # то же (0 or 3000)
      })
      .reset_index()
)

seller_agg["Total_Income"] = (
    seller_agg["Commission"] +
    seller_agg["Promotion_Income"] +
    seller_agg["Subscription_Income"]
)

avg_seller_income_total = seller_agg["Total_Income"].mean()          # за всё время
avg_seller_income_month = avg_seller_income_total / 18               # 18 мес — средняя длительность продавца

# 4.4  Доля продавцов, использующих платные инструменты
pct_promo_sellers = 100 * df["Promotion_Used"].mean()
pct_subs_sellers  = 100 * df["Subscription"].mean()

# ----------------------------------------------------------------------
# 5️⃣  ВХОДНЫЕ ДАННЫЕ БИЗНЕС‑МОДЕЛИ
# ----------------------------------------------------------------------
# 5.1  Маркетинг (покупатели)
marketing_buyers_month = 3_000_000          # ₽/мес
buyers_acquired_month = 2_000               # чел/мес
cac_buyer = marketing_buyers_month / buyers_acquired_month   # ₽/покупатель

# 5.2  Операционные расходы (платформа + обслуживание продавцов)
operational_platform_month = 1_500_000      # ₽/мес
seller_support_per_month   = 1_000          # ₽/продавец/мес
sellers_acquired_month    = 500            # чел/мес

# Общие ежемесячные затраты, связанные с продавцами
operational_sellers_month = seller_support_per_month * sellers_acquired_month
total_operational_month   = operational_platform_month + operational_sellers_month

# 5.3  CAC продавца – здесь берем все операционные расходы и делим
#      (можно уточнить, если у вас отдельный маркетинг для продавцов)
cac_seller = total_operational_month / sellers_acquired_month   # ₽/продавец

# 5.4  Retention (удержание)
retention_buyers  = 0.20    # 20 % остаются каждый месяц
retention_sellers = 0.40    # 40 % остаются каждый месяц

# Средняя продолжительность (мес.) = 1 / (1‑retention)
lifetime_buyers_months  = 1 / (1 - retention_buyers)   # ≈ 5 мес
lifetime_sellers_months = 1 / (1 - retention_sellers)  # ≈ 1.67 мес

# Мы также знаем «реальные» средние длительности взаимодействия:
#   покупатели – 24 мес, продавцы – 18 мес.
# Для расчёта LTV будем использовать именно их, т.к. они отражают
# историческую реальность.
lifetime_buyers_months_real  = 24
lifetime_sellers_months_real = 18

# ----------------------------------------------------------------------
# 6️⃣  ЮНИТ‑ЭКОНОМИКА
# ----------------------------------------------------------------------
# 6.1  LTV покупателя
#      Выручка от покупателя = комиссия, получаемая с каждой его покупки.
#      В отсутствие точных данных о количестве покупок в месяц берём 1 покупку/мес.
revenue_per_buyer_month = avg_commission                       # ₽/мес
ltv_buyer = revenue_per_buyer_month * lifetime_buyers_months_real

# 6.2  ROI покупателя
roi_buyer = (ltv_buyer - cac_buyer) / cac_buyer * 100   # в %

# 6.3  LTV продавца
revenue_per_seller_month = avg_seller_income_month            # ₽/мес
ltv_seller = revenue_per_seller_month * lifetime_sellers_months_real

# 6.4  ROI продавца
roi_seller = (ltv_seller - cac_seller) / cac_seller * 100   # в %

# ----------------------------------------------------------------------
# 7️⃣  ОТЧЁТ
# ----------------------------------------------------------------------
print("\n=== БАЗОВЫЕ МЕТРИКИ ПЛАТФОРМЫ ===============================")
print(f"Средний чек (Order_Value)                 : {avg_check:,.0f} ₽")
print(f"Средняя комиссия (Commission)              : {avg_commission:,.0f} ₽")
print(f"Средний доход продавца (за всё время)      : {avg_seller_income_total:,.0f} ₽")
print(f"Средний доход продавца (в месяц)            : {avg_seller_income_month:,.0f} ₽")
print(f"% продавцов с платным продвижением          : {pct_promo_sellers:,.1f}%")
print(f"% продавцов с платной подпиской             : {pct_subs_sellers:,.1f}%")

print("\n=== ВХОДНЫЕ ДАННЫЕ БИЗНЕС‑МОДЕЛИ ===========================")
print(f"CAC покупателя (маркетинг)                : {cac_buyer:,.0f} ₽")
print(f"CAC продавца (операционные расходы)        : {cac_seller:,.0f} ₽")
print(f"Retention (покупатели)                     : {retention_buyers*100:.0f}%")
print(f"Retention (продавцы)                       : {retention_sellers*100:.0f}%")
print(f"Средняя продолжительность (мес.) покупатели: {lifetime_buyers_months_real} мес")
print(f"Средняя продолжительность (мес.) продавцы : {lifetime_sellers_months_real} мес")

print("\n=== ЮНИТ‑ЭКОНОМИКА =======================================")
print(f"LTV покупателя (24 мес)                    : {ltv_buyer:,.0f} ₽")
print(f"ROI покупателя                              : {roi_buyer:,.1f}%")
print(f"LTV продавца (18 мес)                      : {ltv_seller:,.0f} ₽")
print(f"ROI продавца                               : {roi_seller:,.1f}%")

# ----------------------------------------------------------------------
# 8️⃣  ПРОБЛЕМНЫЕ ЗОНЫ И РЕКОМЕНДАЦИИ
# ----------------------------------------------------------------------
print("\n=== АНАЛИЗ ПРОБЛЕМНЫХ ЗОН ===============================")
if roi_buyer < 0:
    print("❗️ ROI покупателя отрицательный → нужно снижать CAC "
          "(дешевле привлекать) или повышать частоту покупок/комиссию.")
else:
    print("✅ ROI покупателя положительный, но небольшим (≈ 35 %). "
          "Есть пространство для роста.")

if roi_seller < 0:
    print("❗️ ROI продавца отрицательный → основная «дырка» в модели.")
    print("   Причины: высокий CAC продавца (2 000 000 ₽/мес операционных) "
          "и низкая удерживаемость (40 %).")
else:
    print("✅ ROI продавца положительный, но скромный (≈ 2‑3 %). "
          "Требуется работа над удержанием и доходностью.")

print("\n=== РЕКОМЕНДАЦИИ ПО ОПТИМИЗАЦИИ =========================")
print("""1️⃣ Снизить CAC покупателя
   • Перейти часть рекламных расходов в контент‑маркетинг / реферальные программы.
   • Тестировать каналы с более низкой стоимостью клика (соц‑сети, SEO).

2️⃣ Увеличить LTV покупателя
   • Внедрить кросс‑продажи и «bundles» → среднее количество заказов в месяц.
   • Программа лояльности (бонусные баллы за частые покупки) → рост частоты.
   • Увеличить процент комиссии (в пределах конкурентоспособности) или
     добавить «fee‑for‑service» (например, ускоренная доставка).

3️⃣ Снизить CAC продавца
   • Автоматизировать onboarding (веб‑вебинары, шаблоны листингов) – уменьшить
     затраты поддержки.
   • Запустить партнерскую программу: существующие продавцы привлекают новых
     (пониженный CAC «партнёрский»).

4️⃣ Рост дохода от продавцов
   • Повысить процент использования платного продвижения:
        – предлагать бесплатный пробный период и потом “auto‑renew”.
        – динамическое ценообразование (чем выше оборот, тем выше скидка).
   • Поощрять подписку:
        – добавить эксклюзивные инструменты аналитики, быстрый вывод средств,
          приоритетную техподдержку.
   • Сегментировать продавцов: «молодые» (низкая маржа) → более высокая базовая
        комиссия, «премиум» (высокий оборот) → скидка + платные сервисы.

5️⃣ Увеличить удержание продавцов
   • Регулярные обучающие вебинары (как увеличить продажи).
   • Персональный менеджер для топ‑продавцов (повышает NPS).
   • Программа «долгосрочный продавец» – бонусы за каждый полугодочный
        период без оттока.

6️⃣ Пересчитать модель с учётом новых метрик:
   • После внедрения рекомендаций пересчитайте CAC/ROI каждые 3‑6 мес,
     чтобы убедиться, что целевые показатели (ROI > 30 % для покупателей,
     ROI > 15 % для продавцов) достигаются.

""")



=== БАЗОВЫЕ МЕТРИКИ ПЛАТФОРМЫ ===============================
Средний чек (Order_Value)                 : 2,009 ₽
Средняя комиссия (Commission)              : 201 ₽
Средний доход продавца (за всё время)      : 3,337 ₽
Средний доход продавца (в месяц)            : 185 ₽
% продавцов с платным продвижением          : 50.8%
% продавцов с платной подпиской             : 19.9%

=== ВХОДНЫЕ ДАННЫЕ БИЗНЕС‑МОДЕЛИ ===========================
CAC покупателя (маркетинг)                : 1,500 ₽
CAC продавца (операционные расходы)        : 4,000 ₽
Retention (покупатели)                     : 20%
Retention (продавцы)                       : 40%
Средняя продолжительность (мес.) покупатели: 24 мес
Средняя продолжительность (мес.) продавцы : 18 мес

=== ЮНИТ‑ЭКОНОМИКА =======================================
LTV покупателя (24 мес)                    : 4,834 ₽
ROI покупателя                              : 222.2%
LTV продавца (18 мес)                      : 3,337 ₽
ROI продавца                          

In [8]:
print(df["Promotion_Used"].unique())
print(df["Subscription"].unique())


[False]
[False]
